In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack as fft
from astropy.io import fits

# Import fits

In [17]:
def import_fits(file_path):
    """Create an Image class from a FITs file. Can process a 2-D single 
    image data array or a 3-D image datacube.  Return is a single Image
    object or a list of Image objects, respectively."""

    ## Read fits file
    hdulist = fits.open(file_path, memmap=True)
    data = hdulist[0].data

    shape = data.shape

    ## Create Image objects
    if len(shape) == 2:
        return data
    elif len(shape) == 3:
        image_list = []
        
    ## Iterate over datacube and initialize Image objects
        for i in range(data.shape[0]):
            single_image_data = data[i,:,:]
            image_list.append(single_image_data)  
        return image_list
    else:
        print(shape)
        sys.exit("FITs Read Error: Must be 2-D or 3-D Image datacube")

In [23]:
file_name = "/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160227/aored_When_2016.2.27_5.53.56_poldm_phase.fits"

In [25]:
image_list = import_fits(file_name)
single_image = image_list[0]

In [27]:
single_image.shape

(48, 48)